In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from IPython import display
from tensorflow.python.client import device_lib
import os
import re

In [2]:
print(device_lib.list_local_devices())
gpus = tf.config.list_physical_devices('GPU')
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3989939004107434461
xla_global_id: -1
]


In [3]:
def return_DataPathList(path="anno.csv"):
    f = open(path, mode="r", encoding="UTF-8")
    tmp = f.readlines()
    datalist = []
    for dpath in tmp:
        datalist.append(dpath.split(",")[0])
    f.close()
    return datalist

In [4]:
def return_Datalabel(path="anno.csv"):
    f = open(path, mode="r", encoding="UTF-8")
    tmp = f.readlines()
    datalist = []
    
    for dpath in tmp:
        tmp2 = dpath.split(",")[1].replace("\n", "")
        datalist.append([int(i) for i in tmp2])
    return datalist

In [5]:
return_Datalabel()

[[0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1]

In [6]:
datalist = return_DataPathList()
data = [] # 読み込んだ座標データを格納する
label = return_Datalabel()
train_data = [] # 学習データを格納する
test_data = [] #　テストデータを格納する
train_label = [] # 学習ラベルを格納する
test_label = [] # テストラベルを格納する
tmp_data = 0 # 一時的に座標データを格納する
max = 0
for i in range(len(return_Datalabel())):
    tmp_data = pd.read_csv(datalist[i], header=None).values

    data.append(tmp_data)

In [8]:
for i in range(len(data)):
    if i%20 == 0:
        test_data.append(data[i])
        test_label.append(label[i])
    else:
        train_data.append(data[i])
        train_label.append(label[i])
train_data = np.array(train_data).astype(np.float32)
test_data = np.array(test_data).astype(np.float32)
train_label = np.array(train_label).astype(np.int64)
test_label = np.array(test_label).astype(np.int64)
train_data = np.expand_dims(train_data, -1)
test_data = np.expand_dims(test_data, -1)
test_label = keras.utils.to_categorical(test_label, 2)
train_label = keras.utils.to_categorical(train_label, 2)

In [9]:
dim1 = np.array([[1]]*len(train_data)).astype(np.int64)
dim2 = np.array([[1]]*len(test_data)).astype(np.int64)
#print(dim)
print(len(train_data[0]), len(train_data[0][0]))

13 14


In [10]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_label))
test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_label))

In [11]:
BATCH_SIZE = 10
SHUFFLE_BUFFER_SIZE = 1000000
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)
print(train_dataset)

<BatchDataset element_spec=(TensorSpec(shape=(None, 13, 14, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>


In [16]:
def build_model(input_dim, output_dim, rnn_layers=5, rnn_units=128):
    """Model similar to DeepSpeech2."""
    # Model's input
    input_model = layers.Input((input_dim[0], input_dim[1]), name="input")
    # Expand the dimension to use 2D CNN.
    x = layers.Reshape((-1, input_dim[0], 1), name="expand_dim")(input_model)
    x = layers.Conv2D(32, kernel_size=(3, 3), activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(64, kernel_size=(3, 3), activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.5)(x)
    # Classification layer
    output = layers.Dense(units=output_dim, activation="softmax")(x)
    # Model
    model = keras.Model(input_model, output, name="Shogi-winner-model")
    # Optimizer
    opt = keras.optimizers.Adam(learning_rate=1e-6)
    # Compile the model and return
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=["accuracy"])
    return model


model = build_model(
    input_dim=[13,14],
    output_dim=2,
    rnn_units=512
)
model.summary(line_length=110)

Model: "Shogi-winner-model"
______________________________________________________________________________________________________________
 Layer (type)                                    Output Shape                                Param #          
 input (InputLayer)                              [(None, 13, 14)]                            0                
                                                                                                              
 expand_dim (Reshape)                            (None, 14, 13, 1)                           0                
                                                                                                              
 conv2d_2 (Conv2D)                               (None, 12, 11, 32)                          320              
                                                                                                              
 max_pooling2d_2 (MaxPooling2D)                  (None, 6, 5, 32)                   

In [13]:
# A utility function to decode the output of the network
def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    # Use greedy search. For complex tasks, you can use beam search
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0]
    # Iterate over the results and get back the text
    output_text = []
    for result in results:
        result = tf.strings.reduce_join(num_to_char(result)).numpy().decode("utf-8")
        output_text.append(result)
    return output_text


# A callback class to output a few transcriptions during training
class CallbackEval(keras.callbacks.Callback):
    """Displays a batch of outputs after every epoch."""

    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset

    def on_epoch_end(self, epoch: int, logs=None):
        predictions = []
        targets = []
        for batch in self.dataset:
            X, y = batch
            batch_predictions = model.predict(X)
            batch_predictions = decode_batch_predictions(batch_predictions)
            predictions.extend(batch_predictions)
            for label in y:
                label = (
                    tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
                )
                targets.append(label)

        for i in np.random.randint(0, len(predictions), 2):
            print(f"Target    : {targets[i]}")
            print(f"Prediction: {predictions[i]}")
            print("-" * 100)

In [14]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
tensorboard_callback = []

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_label))
#test_dataset = test_dataset.batch()
"""print(len(test_dataset))
for batch in test_dataset:
    X, y = batch
    batch_predictions = model.predict(X)
    batch_predictions = decode_batch_predictions(batch_predictions)
    predictions.extend(batch_predictions)

for i in range(len(predictions)):
    print(f"Target    : {targets[i]}")
    print(f"Prediction: {predictions[i]}")
    print("-" * 100)"""

model.fit(train_dataset, epochs=1000, batch_size=32)

Epoch 1/1000
3722/3722 [==============================] - 9s 2ms/step - loss: 0.9532 - accuracy: 0.5108
Epoch 2/1000
3722/3722 [==============================] - 9s 2ms/step - loss: 0.8916 - accuracy: 0.5081
Epoch 3/1000
3722/3722 [==============================] - 10s 3ms/step - loss: 0.8446 - accuracy: 0.5154
Epoch 4/1000
3722/3722 [==============================] - 11s 3ms/step - loss: 0.8036 - accuracy: 0.5172
Epoch 5/1000
3722/3722 [==============================] - 12s 3ms/step - loss: 0.7790 - accuracy: 0.5192
Epoch 6/1000
3722/3722 [==============================] - 12s 3ms/step - loss: 0.7544 - accuracy: 0.5254
Epoch 7/1000
3722/3722 [==============================] - 11s 3ms/step - loss: 0.7410 - accuracy: 0.5276
Epoch 8/1000
3722/3722 [==============================] - 11s 3ms/step - loss: 0.7261 - accuracy: 0.5322
Epoch 9/1000
3722/3722 [==============================] - 11s 3ms/step - loss: 0.7160 - accuracy: 0.5392
Epoch 10/1000
3722/3722 [==============================] 